1. Dataset “Babyboom” (переменные Time of birth recorded on the 24-hour clock, Sex of the child (1 = girl, 2 = boy), Birth weight in grams,      Number of minutes after midnight of each birth):

· Проверьте гипотезу, если средний вес девочек такой же, как вес мальчиков.

· Проверьте гипотезу, если дисперсия веса девочек такая же, как и веса мальчиков.

In [2]:
import pandas as pd
import scipy.stats as stats


# Загрузка данных
file_path = 'D:/python/matstat/me/lab_3/Datasets/babyboom.dat.txt'
columns = ['Time_of_birth', 'Sex', 'Birth_weight', 'Minutes_after_midnight']
data = pd.read_csv(file_path, sep='\s+', names=columns)

# Разделение данных по полу
girls_weight = data[data['Sex'] == 1]['Birth_weight']
boys_weight = data[data['Sex'] == 2]['Birth_weight']

stat, p = stats.shapiro(girls_weight)
print(stat, p)
if p > 0.05:
    print('распределение нормальное')
else:
    print('распределение ненормальное')


#Т.к распределение девочек ненормальное
# Гипотеза 1: Проверка, равна ли дисперсия веса девочек дисперсии веса мальчиков
f_stat, p_value_var = stats.levene(girls_weight, boys_weight)
print(f"Проверка дисперсии: F-статистика = {f_stat}, p-значение = {p_value_var}")   

# Гипотеза 2: Проверка, равен ли средний вес девочек среднему весу мальчиков
t_stat, p_value_mean = stats.ttest_ind(boys_weight, girls_weight, equal_var=False)
print(f"Проверка среднего веса: Тест Уэлча = {t_stat}, p-значение = {p_value_mean}")


# Интерпретация
if p_value_mean < 0.05:
    print("Отвергаем нулевую гипотезу для средних весов: Есть значительная разница в среднем весе между девочками и мальчиками.")
else:
    print("Не отвергаем нулевую гипотезу для средних весов: Нет значительной разницы в среднем весе между девочками и мальчиками.")

if p_value_var < 0.05:
    print("Отвергаем нулевую гипотезу для дисперсий: Есть значительная разница в дисперсии веса между девочками и мальчиками.")
else:
    print("Не отвергаем нулевую гипотезу для дисперсий: Нет значительной разницы в дисперсии веса между девочками и мальчиками.")


0.8702831268310547 0.017984924837946892
распределение ненормальное
Проверка дисперсии: F-статистика = 1.8154103472308154, p-значение = 0.18508483634639294
Проверка среднего веса: Тест Уэлча = 1.4210557069307492, p-значение = 0.1664888203268482
Не отвергаем нулевую гипотезу для средних весов: Нет значительной разницы в среднем весе между девочками и мальчиками.
Не отвергаем нулевую гипотезу для дисперсий: Нет значительной разницы в дисперсии веса между девочками и мальчиками.


2. Dataset “Euroweight” (переменные weight, batch):

· Проверить гипотезы о том, что среднее значение веса монеты одинаково в разных пакетах (попарно и все вместе).

In [6]:
import pandas as pd
import itertools
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Загрузка данных
file_path = 'D:/python/matstat/me/lab_3/Datasets/euroweight.dat.txt'
columns = ['ID', 'weight', 'batch']
data = pd.read_csv(file_path, sep='\s+', names=columns)

# Проверка на нормальность распределения

normal_batches = []

for batch in data["batch"].unique():
    print(f"Пакет {batch}: ", end="")
    stat, p = stats.shapiro(data[data["batch"] == batch]["weight"])

    if p > 0.05:
        print("нормальное распределение")
        normal_batches.append(batch)
    else:
        print("не нормальное распределение")
print()
#print(len(normal_batches))
#print(len(data["batch"].unique()))
# Проверка гипотезы о равенстве средних значений веса монет во всех пакетах (ANOVA или Краскела–Уоллиса)
batches = [data[data["batch"] == batch]["weight"] for batch in data["batch"].unique()]
#print(batches)

# Проверяем равенство дисперсий для всех пакетов (тест Левена)
levene_stat, levene_p = stats.levene(*batches)

if len(normal_batches) == len(data["batch"].unique()) and levene_p > 0.05:
    # Все пакеты нормальные и имеют равные дисперсии — используем ANOVA
    anova_result = stats.f_oneway(*batches)
    print("Результаты ANOVA:", anova_result)
    if anova_result.pvalue < 0.05:
        print("Есть статистически значимые различия между средними значениями весов пакетов.")
        # Попарные сравнения с помощью теста Тьюки
        tukey_result = pairwise_tukeyhsd(data["weight"], data["batch"])
        print("Результаты попарного теста Тьюки:\n", tukey_result)
else:
    # Либо есть пакеты с ненормальным распределением, либо дисперсии не равны
    # Используем тест Краскела–Уоллиса как аналог ANOVA для сравнения всех пакетов
    kruskal_result = stats.kruskal(*batches)
    print("Результаты теста Краскела–Уоллиса:", kruskal_result)
    if kruskal_result.pvalue < 0.05:
        print("Есть статистически значимые различия между средними значениями весов пакетов.")
        # Попарные сравнения тестом Манна–Уитни для пакетов, у которых хотя бы один не нормальный или неравные дисперсии
        print("\nПопарные сравнения:")
        pairs = itertools.combinations(data["batch"].unique(), 2)
        for batch1, batch2 in pairs:
            data1 = data[data["batch"] == batch1]["weight"]
            data2 = data[data["batch"] == batch2]["weight"]

            if batch1 in normal_batches and batch2 in normal_batches:
                # Если обе выборки нормальные
                levene_stat, levene_p = stats.levene(data1, data2)
                if levene_p > 0.05:
                    # Используем стандартный t-тест при равных дисперсиях
                    ttest_result = stats.ttest_ind(data1, data2, equal_var=True)
                    print(f"Пакет {batch1} vs Пакет {batch2} (t-тест): p-value = {ttest_result.pvalue}")
                else:
                    # Используем тест Уэлча при неравных дисперсиях
                    welch_result = stats.ttest_ind(data1, data2, equal_var=False)
                    print(f"Пакет {batch1} vs Пакет {batch2} (тест Уэлча): p-value = {welch_result.pvalue}")
            else:
                # Если хотя бы одна из выборок ненормальная — используем тест Манна–Уитни
                mannwhitney_result = stats.mannwhitneyu(data1, data2, alternative="two-sided")
                print(f"Пакет {batch1} vs Пакет {batch2} (тест Манна–Уитни): p-value = {mannwhitney_result.pvalue}")


Пакет 1: нормальное распределение
Пакет 2: нормальное распределение
Пакет 3: не нормальное распределение
Пакет 4: нормальное распределение
Пакет 5: нормальное распределение
Пакет 6: не нормальное распределение
Пакет 7: нормальное распределение
Пакет 8: не нормальное распределение

Результаты теста Краскела–Уоллиса: KruskalResult(statistic=97.49984084853487, pvalue=3.53927567965911e-18)
Есть статистически значимые различия между средними значениями весов пакетов.

Попарные сравнения:
Пакет 1 vs Пакет 2 (t-тест): p-value = 0.26147780679017946
Пакет 1 vs Пакет 3 (тест Манна–Уитни): p-value = 0.0024542093593894346
Пакет 1 vs Пакет 4 (тест Уэлча): p-value = 7.269213938290278e-05
Пакет 1 vs Пакет 5 (тест Уэлча): p-value = 5.01571601022176e-05
Пакет 1 vs Пакет 6 (тест Манна–Уитни): p-value = 0.09596023756227035
Пакет 1 vs Пакет 7 (t-тест): p-value = 0.26532250649811384
Пакет 1 vs Пакет 8 (тест Манна–Уитни): p-value = 0.19324501318679987
Пакет 2 vs Пакет 3 (тест Манна–Уитни): p-value = 1.17941

3. Dataset “iris.txt” (прочитайте описание данных в файле «iris_description.txt», переменные sepal length, sepal width, petal length, petal width, class):

o Проверить гипотезы о равенстве распределений характеристик цветков разных типов.

o Проверьте гипотезы о равенстве средних и дисперсий различных характеристик цветов разных типов.

In [13]:
import pandas as pd
import scipy.stats as stats
import itertools
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Загрузка данных
iris = pd.read_csv("D:/python/matstat/me/lab_2/Datasets/iris.txt",
                    header=None, 
                    names=["sepal_length", "sepal_width", "petal_length", "petal_width", "class"])

# Определение классов и признаков
classes = iris["class"].unique()
#print(classes)
features = ["sepal_length", "sepal_width", "petal_length", "petal_width"]

# Проверка распределений с помощью теста Краскела–Уоллиса
print("Проверка распределений характеристик по тесту Краскела–Уоллиса:")
for feature in features:
    groups = [iris[iris["class"] == cls][feature] for cls in classes]
    #print(f'group:{groups}')
    kruskal_test = stats.kruskal(*groups)
    print(f"{feature}: p-value = {kruskal_test.pvalue}")
    if kruskal_test.pvalue < 0.05:
        print(f"Распределения {feature} значимо отличаются между классами.\n")
    else:
        print(f"Распределения {feature} не имеют значимых различий между классами.\n")

# Проверка нормальности и равенства дисперсий
for feature in features:
    print(f"\nАнализ признака '{feature}'\n")
    normal_distributions = []
    
    # Проверка на нормальность
    for cls in classes:
        w_stat, p_val = stats.shapiro(iris[iris["class"] == cls][feature])
        if p_val > 0.05:
            normal_distributions.append(cls)
        print(f"Класс {cls}, проверка на нормальность (p-value = {p_val}): {'нормальное' if p_val > 0.05 else 'не нормальное'}")

    # Проверка равенства дисперсий
    groups = [iris[iris["class"] == cls][feature] for cls in classes]
    levene_stat, levene_p_val = stats.levene(*groups)
    print(f"\nТест Левена для {feature}: p-value = {levene_p_val}")
    
    # Проверка равенства средних значений (ANOVA или Краскела–Уоллиса)
    if len(normal_distributions) == len(classes) and levene_p_val > 0.05:
        # Если распределения нормальные и дисперсии равны, используем ANOVA
        anova_test = stats.f_oneway(*groups)
        print(f"ANOVA для {feature}: p-value = {anova_test.pvalue}")
        if anova_test.pvalue < 0.05:
            print(f"Средние значения {feature} значимо отличаются между классами.\n")
        # Попарные сравнения с использованием теста Тьюки
        tukey_test = pairwise_tukeyhsd(iris[feature], iris["class"])
        print("Результаты попарного теста Тьюки:\n", tukey_test)
    else:
        # Если условия ANOVA не выполняются, используем тест Краскела–Уоллиса
        kruskal_test = stats.kruskal(*groups)
        print(f"Тест Краскела–Уоллиса для {feature}: p-value = {kruskal_test.pvalue}")
        if kruskal_test.pvalue < 0.05:
            print(f"Средние значения {feature} значимо отличаются между классами.")

        # Попарные сравнения
        print("\nПопарные сравнения:")
        for cls1, cls2 in itertools.combinations(classes, 2):
            data1 = iris[iris["class"] == cls1][feature]
            data2 = iris[iris["class"] == cls2][feature]
            
            if cls1 in normal_distributions and cls2 in normal_distributions:
                levene_stat, levene_p_val = stats.levene(data1, data2)
                if levene_p_val > 0.05:
                    ttest_res = stats.ttest_ind(data1, data2, equal_var=True)
                    print(f"{cls1} vs {cls2} (t-test): p-value = {ttest_res.pvalue}")
                else:
                    welch_res = stats.ttest_ind(data1, data2, equal_var=False)
                    print(f"{cls1} vs {cls2} (Welch's t-test): p-value = {welch_res.pvalue}")
            else:
                mannwhitney_res = stats.mannwhitneyu(data1, data2, alternative="two-sided")
                print(f"{cls1} vs {cls2} (Mann–Whitney test): p-value = {mannwhitney_res.pvalue}")


Проверка распределений характеристик по тесту Краскела–Уоллиса:
sepal_length: p-value = 8.91873433246198e-22
Распределения sepal_length значимо отличаются между классами.

sepal_width: p-value = 2.6882119006774528e-14
Распределения sepal_width значимо отличаются между классами.

petal_length: p-value = 4.7967237479455454e-29
Распределения petal_length значимо отличаются между классами.

petal_width: p-value = 3.415387999117524e-29
Распределения petal_width значимо отличаются между классами.


Анализ признака 'sepal_length'

Класс Iris-setosa, проверка на нормальность (p-value = 0.4595010578632355): нормальное
Класс Iris-versicolor, проверка на нормальность (p-value = 0.4647378921508789): нормальное
Класс Iris-virginica, проверка на нормальность (p-value = 0.25831347703933716): нормальное

Тест Левена для sepal_length: p-value = 0.0022585277836218586
Тест Краскела–Уоллиса для sepal_length: p-value = 8.91873433246198e-22
Средние значения sepal_length значимо отличаются между классами.

П

4. Dataset “sugery.xlsx” (переменные «До операции, V left», «До операции, V right», «После операции, V left», «После операции, V right»):

· Проверить гипотезу об успешности операции с вероятностью 0.7 (0.8). Под успехом мы подразумеваем, что «V справа» до операции меньше, чем «V справа» после операции, и одновременно с этим «V слева» до операции меньше, чем «V слева» после операции.

In [2]:
import pandas as pd
from scipy.stats import binom_test

# Загрузка данных
data = pd.read_excel("D:/python/matstat/me/lab_3/Datasets/surgery.xlsx", skiprows=1)

print("Названия столбцов:", data.columns)

# Определение успешных случаев
data['success'] = (data['V right.1'] > data['V right']) & \
                  (data['V left.1'] > data['V left'])

# Подсчет числа успешных операций и общего количества операций
success_count = data['success'].sum()
print(success_count)
total_count = len(data)
print(success_count/total_count)

# Параметры гипотезы
# Проверяем гипотезу с вероятностью успеха 0.7
p_hypothesis_0_7 = 0.7
p_value_0_7 = binom_test(success_count, total_count, p=p_hypothesis_0_7, alternative='two-sided')

# Проверяем гипотезу с вероятностью успеха 0.8
p_hypothesis_0_8 = 0.8
p_value_0_8 = binom_test(success_count, total_count, p=p_hypothesis_0_8, alternative='two-sided')

# Вывод результатов
print(f"Гипотеза с вероятностью успеха 0.7: p-значение = {p_value_0_7}")
if p_value_0_7 < 0.05:
    print("Отклоняем гипотезу о вероятности успеха 0.7.")
else:
    print("Не отклоняем гипотезу о вероятности успеха 0.7.")

print(f"\nГипотеза с вероятностью успеха 0.8: p-значение = {p_value_0_8}")
if p_value_0_8 < 0.05:
    print("Отклоняем гипотезу о вероятности успеха 0.8.")
else:
    print("Не отклоняем гипотезу о вероятности успеха 0.8.")


Названия столбцов: Index(['V right', 'V left', 'V right.1', 'V left.1'], dtype='object')
69
0.7340425531914894
Гипотеза с вероятностью успеха 0.7: p-значение = 0.5018300653842115
Не отклоняем гипотезу о вероятности успеха 0.7.

Гипотеза с вероятностью успеха 0.8: p-значение = 0.12101781363892931
Не отклоняем гипотезу о вероятности успеха 0.8.


C:\Users\Евгений\AppData\Local\Temp\ipykernel_18760\3526852375.py:22: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_value_0_7 = binom_test(success_count, total_count, p=p_hypothesis_0_7, alternative='two-sided')
C:\Users\Евгений\AppData\Local\Temp\ipykernel_18760\3526852375.py:26: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_value_0_8 = binom_test(success_count, total_count, p=p_hypothesis_0_8, alternative='two-sided')
